In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

d:\App\anaconda\envs\toolkit\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#movies data
movies = tfds.load("movielens/100k-movies", split="train")
#ratings data
ratings = tfds.load("movielens/100k-ratings", split="train")

In [20]:
ratings_df = tfds.as_dataframe(ratings)

In [39]:
movies_df = tfds.as_dataframe(movies)
for x in movies_df[:3].iterrows():
    print(x)

,movie_genres,movie_id,movie_title
0,[4],b'1681',b'You So Crazy (1994)'
1,"[4, 7]",b'1457',b'Love Is All There Is (1996)'
2,"[1, 3]",b'500',b'Fly Away Home (1996)'


In [27]:
def sort_ratings(ds):
    timestamps = []
    features_list = []

    for example in ds:
        timestamps.append(example['timestamp'])
        features_list.append(example)

    # convert list to tensor
    timestamps = tf.convert_to_tensor(timestamps)
    features_list = tf.stack(features_list)
    # sort index
    sorted_indices = tf.argsort(timestamps)
    # sort all data
    sorted_features = tf.gather(ds, sorted_indices)
    return tf.data.Dataset.from_tensor_slices(sorted_features)

In [31]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [32]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [52]:
movie_titles = movies.batch(2_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

In [83]:
unique_movie_titles = np.unique(list(movie_titles))
unique_user_ids = np.unique(list(user_ids))

unique_user_ids[:10]

array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
       b'106', b'107'], dtype=object)

In [54]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [55]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [56]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

In [57]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [61]:
from typing import Dict, Text

class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False):
    # pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # and pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [62]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [63]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [64]:
model.fit(cached_train, epochs=5)

Epoch 1/5
10/10 [==============================] - 6s 399ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0093 - factorized_top_k/top_10_categorical_accuracy: 0.0203 - factorized_top_k/top_50_categorical_accuracy: 0.1000 - factorized_top_k/top_100_categorical_accuracy: 0.1785 - loss: 69866.8345 - regularization_loss: 0.0000e+00 - total_loss: 69866.8345
Epoch 2/5
10/10 [==============================] - 4s 393ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0032 - factorized_top_k/top_5_categorical_accuracy: 0.0206 - factorized_top_k/top_10_categorical_accuracy: 0.0400 - factorized_top_k/top_50_categorical_accuracy: 0.1703 - factorized_top_k/top_100_categorical_accuracy: 0.2924 - loss: 67492.0085 - regularization_loss: 0.0000e+00 - total_loss: 67492.0085
Epoch 3/5
10/10 [==============================] - 4s 388ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0035 - factorized_top_k/top_5_categorical_accuracy: 

In [65]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 154ms/step - factorized_top_k/top_1_categorical_accuracy: 5.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0055 - factorized_top_k/top_10_categorical_accuracy: 0.0147 - factorized_top_k/top_50_categorical_accuracy: 0.1061 - factorized_top_k/top_100_categorical_accuracy: 0.2182 - loss: 31184.7298 - regularization_loss: 0.0000e+00 - total_loss: 31184.7298


{'factorized_top_k/top_1_categorical_accuracy': 0.0005000000237487257,
 'factorized_top_k/top_5_categorical_accuracy': 0.005549999885261059,
 'factorized_top_k/top_10_categorical_accuracy': 0.014650000259280205,
 'factorized_top_k/top_50_categorical_accuracy': 0.10610000044107437,
 'factorized_top_k/top_100_categorical_accuracy': 0.21819999814033508,
 'loss': 28321.052734375,
 'regularization_loss': 0,
 'total_loss': 28321.052734375}

In [77]:
model.user_model.weights

[<tf.Variable 'embedding/embeddings:0' shape=(944, 32) dtype=float32, numpy=
 array([[ 3.55528332e-02,  2.19892971e-02, -9.03975964e-03, ...,
          1.40443705e-02,  2.37191208e-02,  1.78357475e-02],
        [ 7.29114711e-01, -1.92525759e-01, -1.95666432e-01, ...,
          1.25993431e-01, -9.01168063e-02,  3.27466100e-01],
        [-3.74643743e-01,  6.77619427e-02,  3.46409798e-01, ...,
         -2.00624019e-01,  3.63519825e-02, -5.65037429e-01],
        ...,
        [-8.34141672e-03,  3.46097916e-01,  6.23958468e-01, ...,
         -8.55518281e-01,  4.19525087e-01, -3.38960409e-01],
        [ 6.89618140e-02,  4.14748222e-01,  4.39767241e-02, ...,
         -4.08544362e-01,  8.06384720e-04, -2.01640323e-01],
        [-5.25799751e-01, -2.75419295e-01,  1.01322383e-01, ...,
          2.14048401e-01, -3.26329619e-01,  4.70659018e-01]], dtype=float32)>]

In [82]:
# create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  movies.batch(128).map(lambda title: (title, model.movie_model(title)))
)

# get recommendations.
_, titles = index(np.array(["21"]), k=3)
print(f"Recommendations for user 13: {titles[0]}")

Recommendations for user 13: [b'Believers, The (1987)' b'Hellraiser: Bloodline (1996)'
 b'Screamers (1995)']
